In [12]:
%load_ext autoreload
%autoreload 2

import os
# avoid thread conflicts between numpy and dask
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

from polsarpro.dev.devtools import parse_psp_parameter_string
import numpy as np
from pathlib import Path
import xarray as xr
from polsarpro.io import read_psp_bin, read_T3_psp, open_netcdf_beam
from polsarpro.decompositions import tsvm
from polsarpro.util import plot_h_alpha_plane
from polsarpro.dev.metrics import summarize_metrics
import shutil

# optional import for progress bar
from dask.diagnostics import ProgressBar

# change to your local C-PolSARpro install dir
c_psp_dir = "/home/c_psp/Soft/bin/"
os.environ["PATH"]+=os.pathsep+f"{c_psp_dir}/data_process_sngl/"
os.environ["PATH"]+=os.pathsep+f"{c_psp_dir}/data_convert/"

# change to your data paths
# original dataset
input_alos_data = Path("/data/psp/test_files/SAN_FRANCISCO_ALOS1_slc.nc")

# input files from C
input_test_dir = Path("/data/psp/SAN_FRANCISCO_ALOS1/")

# output files from C
output_test_dir = Path("/data/psp/res/tsvm_c/")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Run the C-version on some test data

In [10]:
# fl1: alpha, phi,tau, psi 
# fl2: alphas
# fl3: phis
# fl4: taus 
# fl5: psis

if not os.path.exists(output_test_dir):
    os.makedirs(output_test_dir)

input_str= f"""id: {input_test_dir} 
od: {output_test_dir}
iodf: S2
nwr: 7
nwc: 7
ofr: 0
ofc: 0
fnr: 18432
fnc: 1248
fl1: 1
fl2: 1
fl3: 1
fl4: 1
fl5: 1
errf: /tmp/MemoryAllocError.txt
mask: {input_test_dir}/mask_valid_pixels.bin
"""
result = parse_psp_parameter_string(input_str)
os.system(f"tsvm_decomposition.exe {result}")

# CPSP does not create another config file in output dir
shutil.copy(input_test_dir / "config.txt", output_test_dir)

'/data/psp/res/tsvm_c/config.txt'

## Load ALOS data and C outputs

In [11]:
# uncomment to test on S matrix made with SNAP
S = open_netcdf_beam(input_alos_data)
out_names = ("psi", "psis")

## Apply the decomposition

In [16]:
file_out = "/data/psp/res/test_tsvm.nc"
# netcdf writer cannot overwrite
if os.path.isfile(file_out):
    os.remove(file_out)

with ProgressBar():
    # uncomment to compute and store result in the notebook
    # out_py = h_a_alpha(S, boxcar_size=[7, 7], flags=out_names).compute()

    # uncomment to compute and store result in a file
    tsvm(S, boxcar_size=[7, 7], flags=out_names).to_netcdf(file_out)
    
    # uncomment to compute and store result in a file (T3 inputs)
    # h_a_alpha(T3, boxcar_size=[7, 7], flags=out_names).to_netcdf(file_out)

[########################################] | 100% Completed | 78.26 s


# Numerical evaluation

In [17]:
# open python output (comment if using compute)
out_py = xr.open_dataset("/data/psp/res/test_tsvm.nc")
# open C-PolSARPro outputs
out_c = {}
# for name in out_names:
for name in ["psi"]:
    file_name = output_test_dir / f"TSVM_{name}.bin"
    out_c[name] = read_psp_bin(file_name)

df = summarize_metrics(out_py, out_c, short_titles=False, verbose=False)
df

/polsarpro-dev/polsarpro/dev/metrics.py:79: UserWarning: Skipping variable 'psis'! Not found in C outputs.
  warnings.warn(f"Skipping variable '{var}'! Not found in C outputs.")


,Difference of Means,Root Mean Square Error,Mean Error (bias),Mean Absolute Error,Mean Normalized Absolute Error,99% percentile of the Normalized Absolute Error,90% percentile of the Normalized Absolute Error,50% percentile of the Normalized Absolute Error
psi,9.589201,57.367069,9.589201,48.234776,0.848729,1.0,1.0,1.0


In [ ]:
# uncomment to test H/Alpha plane histogram
# ax, fig = plot_h_alpha_plane(out_py, bins=500, min_pts=2)
# fig.savefig(output_test_dir / "h_a_alpha_plane.png")